In [1]:
import sys; sys.path.append("..") 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.utility import loadfile, savefile, exists
from src.dataprocessing import *
from src import rnn

## load data

In [2]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-5", "LT-3061-2"]
process_var = input_var + output_var

In [3]:
min_len = 100
continuous_path = './data/3_continuous'

cts_list = []
i = 1
while exists(f"{continuous_path}/cts_{min_len}/dataset {min_len}_{i}.csv"):
    cts_df = loadfile(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv')
    cts_list.append(cts_df)
    i += 1

csv file is loaded from ./data/3_continuous/cts_100/dataset 100_1.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_2.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_3.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_4.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_5.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_6.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_7.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_8.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_9.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_10.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_11.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_12.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_13.csv
csv file is loaded from ./data/3_continuous/cts_100/dataset 100_14.csv
csv file is loa

# iteration

In [5]:
target_list = cts_list

# history size and future size
history_size = 40
future_size = 10
step = 1

# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# data to series
history_series = []
future_series = []
for i in range(len(target_list)):
    history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                step, start_idx=0, end_idx=None)
    if not i:
        history_series = history
        future_series = future
    else:
        history_series = np.concatenate([history_series, history], axis=0)
        future_series = np.concatenate([future_series, future], axis=0)
        
# supervised attention factor
delta_list = [0.5, 1, 1.5, 2, 2.5]
att_list = ['linear', 'exp']
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers = 1
num_neurons = 50
dense_layers = 1
dense_neurons = 50
model_type = 'datt_seq2seq_gru'

for iteration in range(1,11):
    for att_type in att_list:
        for delta in delta_list:
            print(f"{iteration}th iteration")
            print(f"att_type: {att_type}")
            print(f"delta: {delta}")
            factor = rnn.super_attention(delta, future_size, future_num, att_type)
            # Dual-attention Seq2Seq model
            DATT_seq2seq_GRU = rnn.RNN(history_series, history_var, future_series, future_var)
            # TT split
            DATT_seq2seq_GRU.train_test(test_size=test_size, test_num=test_num)
            # TV split
            valid_size = DATT_seq2seq_GRU.history_test.shape[0]/DATT_seq2seq_GRU.history_train.shape[0]
            DATT_seq2seq_GRU.train_valid(valid_size=valid_size)
            # scaling
            DATT_seq2seq_GRU.scaling()
            # modeling
            DATT_seq2seq_GRU.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
            # training
            model_num = iteration
            model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{att_type}_{delta}_{model_num}"
            if not exists(f"./model/{model_name}.h5"):
                DATT_seq2seq_GRU.train()
                DATT_seq2seq_GRU.save_model(f"./model/{model_name}", 'weights')
                
            else:
                DATT_seq2seq_GRU.model.load_weights(f"./model/{model_name}.h5")
            # test
            test_result = DATT_seq2seq_GRU.test()
            if not exists(f'./result/{model_name}.csv', ):
                savefile(test_result, './result', model_name)
            print("\n")
print('end opitmization')

1th iteration
att_type: linear
delta: 0.5
Epoch 1/10000
568/568 - 8s - loss: 0.1751 - val_loss: 0.1265 - 8s/epoch - 14ms/step
Epoch 2/10000
568/568 - 4s - loss: 0.1062 - val_loss: 0.1074 - 4s/epoch - 8ms/step
Epoch 3/10000
568/568 - 4s - loss: 0.0972 - val_loss: 0.0992 - 4s/epoch - 8ms/step
Epoch 4/10000
568/568 - 4s - loss: 0.0908 - val_loss: 0.0942 - 4s/epoch - 8ms/step
Epoch 5/10000
568/568 - 5s - loss: 0.0846 - val_loss: 0.0863 - 5s/epoch - 8ms/step
Epoch 6/10000
568/568 - 4s - loss: 0.0789 - val_loss: 0.0916 - 4s/epoch - 8ms/step
Epoch 7/10000
568/568 - 4s - loss: 0.0747 - val_loss: 0.0802 - 4s/epoch - 8ms/step
Epoch 8/10000
568/568 - 4s - loss: 0.0722 - val_loss: 0.0769 - 4s/epoch - 8ms/step
Epoch 9/10000
568/568 - 4s - loss: 0.0684 - val_loss: 0.0745 - 4s/epoch - 8ms/step
Epoch 10/10000
568/568 - 5s - loss: 0.0666 - val_loss: 0.0738 - 5s/epoch - 8ms/step
Epoch 11/10000
568/568 - 4s - loss: 0.0690 - val_loss: 0.0742 - 4s/epoch - 8ms/step
Epoch 12/10000
568/568 - 4s - loss: 0.0636

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.872774,0.921883,0.950246,0.914968,5.788357,4.91933,3.434282,4.713989
1,0.786074,0.846568,0.899179,0.84394,7.459573,6.895039,4.889402,6.414671
2,0.665648,0.781707,0.842903,0.763419,9.199833,8.225004,6.104141,7.842993
3,0.521835,0.732957,0.80655,0.687114,10.855969,9.098476,6.774535,8.90966
4,0.374648,0.694266,0.793697,0.62087,12.292268,9.736594,6.996429,9.675097
5,0.238705,0.664693,0.794093,0.56583,13.430297,10.197666,6.990474,10.206145
6,0.122796,0.64435,0.798881,0.522009,14.277653,10.502336,6.909943,10.563311
7,0.026628,0.628926,0.801927,0.485827,15.00352,10.726854,6.859397,10.863257
8,-0.048126,0.613698,0.798939,0.454837,15.587192,10.943495,6.913141,11.147943


csv file is saved to: ./result/40_10_1_50_1_50_datt_seq2seq_gru_linear_0.5_1.csv


1th iteration
att_type: linear
delta: 1
Epoch 1/10000
568/568 - 8s - loss: 0.2276 - val_loss: 0.1672 - 8s/epoch - 14ms/step
Epoch 2/10000
568/568 - 5s - loss: 0.1472 - val_loss: 0.1557 - 5s/epoch - 8ms/step
Epoch 3/10000
568/568 - 4s - loss: 0.1349 - val_loss: 0.1477 - 4s/epoch - 7ms/step
Epoch 4/10000
568/568 - 4s - loss: 0.1260 - val_loss: 0.1263 - 4s/epoch - 8ms/step
Epoch 5/10000
568/568 - 4s - loss: 0.1167 - val_loss: 0.1175 - 4s/epoch - 8ms/step
Epoch 6/10000
568/568 - 4s - loss: 0.1080 - val_loss: 0.1221 - 4s/epoch - 8ms/step
Epoch 7/10000
568/568 - 4s - loss: 0.1044 - val_loss: 0.1097 - 4s/epoch - 8ms/step
Epoch 8/10000
568/568 - 4s - loss: 0.0983 - val_loss: 0.1086 - 4s/epoch - 8ms/step
Epoch 9/10000
568/568 - 4s - loss: 0.0968 - val_loss: 0.1062 - 4s/epoch - 8ms/step
Epoch 10/10000
568/568 - 4s - loss: 0.0952 - val_loss: 0.1014 - 4s/epoch - 8ms/step
Epoch 11/10000
568/568 - 4s - loss: 0.0897 - 

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.885802,0.920512,0.953405,0.919906,5.483981,4.962325,3.323485,4.58993
1,0.780956,0.833581,0.922984,0.84584,7.548277,7.180922,4.273369,6.334189
2,0.669314,0.771304,0.88072,0.773779,9.149262,8.418703,5.318941,7.628969
3,0.554182,0.730806,0.84745,0.710813,10.482348,9.135045,6.015911,8.544435
4,0.439755,0.701515,0.8243,0.65519,11.634797,9.620469,6.456683,9.237316
5,0.336294,0.67853,0.810142,0.608322,12.539977,9.985037,6.712525,9.745847
6,0.246672,0.657672,0.799102,0.567816,13.231185,10.303763,6.906133,10.147027
7,0.168008,0.639352,0.785641,0.531,13.871176,10.575088,7.135818,10.527361
8,0.100337,0.622099,0.768076,0.496837,14.441133,10.823858,7.424787,10.896592


csv file is saved to: ./result/40_10_1_50_1_50_datt_seq2seq_gru_linear_1_1.csv


1th iteration
att_type: linear
delta: 1.5
Epoch 1/10000
568/568 - 8s - loss: 0.3258 - val_loss: 0.2250 - 8s/epoch - 14ms/step
Epoch 2/10000
568/568 - 5s - loss: 0.2005 - val_loss: 0.2034 - 5s/epoch - 8ms/step
Epoch 3/10000
568/568 - 4s - loss: 0.1822 - val_loss: 0.1865 - 4s/epoch - 7ms/step
Epoch 4/10000
568/568 - 4s - loss: 0.1683 - val_loss: 0.1694 - 4s/epoch - 7ms/step
Epoch 5/10000
568/568 - 4s - loss: 0.1562 - val_loss: 0.1633 - 4s/epoch - 7ms/step
Epoch 6/10000
568/568 - 4s - loss: 0.1457 - val_loss: 0.1492 - 4s/epoch - 7ms/step
Epoch 7/10000
568/568 - 4s - loss: 0.1390 - val_loss: 0.1422 - 4s/epoch - 8ms/step
Epoch 8/10000
568/568 - 4s - loss: 0.1333 - val_loss: 0.1437 - 4s/epoch - 7ms/step
Epoch 9/10000
568/568 - 4s - loss: 0.1293 - val_loss: 0.1356 - 4s/epoch - 7ms/step
Epoch 10/10000
568/568 - 4s - loss: 0.1249 - val_loss: 0.1423 - 4s/epoch - 7ms/step
Epoch 11/10000
568/568 - 4s - loss: 0.1239 - 

...

csv file is saved to: ./result/40_10_1_50_1_50_datt_seq2seq_gru_exp_1.5_10.csv


10th iteration
att_type: exp
delta: 2
Epoch 1/10000
568/568 - 8s - loss: 0.3835 - val_loss: 0.2395 - 8s/epoch - 14ms/step
Epoch 2/10000
568/568 - 5s - loss: 0.2135 - val_loss: 0.2258 - 5s/epoch - 8ms/step
Epoch 3/10000
568/568 - 4s - loss: 0.1914 - val_loss: 0.1878 - 4s/epoch - 8ms/step
Epoch 4/10000
568/568 - 4s - loss: 0.1739 - val_loss: 0.1715 - 4s/epoch - 8ms/step
Epoch 5/10000
568/568 - 4s - loss: 0.1607 - val_loss: 0.1622 - 4s/epoch - 8ms/step
Epoch 6/10000
568/568 - 4s - loss: 0.1498 - val_loss: 0.1524 - 4s/epoch - 8ms/step
Epoch 7/10000
568/568 - 4s - loss: 0.1422 - val_loss: 0.1528 - 4s/epoch - 8ms/step
Epoch 8/10000
568/568 - 4s - loss: 0.1386 - val_loss: 0.1450 - 4s/epoch - 7ms/step
Epoch 9/10000
568/568 - 4s - loss: 0.1330 - val_loss: 0.1490 - 4s/epoch - 8ms/step
Epoch 10/10000
568/568 - 4s - loss: 0.1289 - val_loss: 0.1373 - 4s/epoch - 8ms/step
Epoch 11/10000
568/568 - 4s - loss: 0.1241 - val_

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.946819,0.917656,0.953841,0.939439,3.742343,5.050663,3.30791,4.033639
1,0.874753,0.841294,0.916353,0.877467,5.707766,7.012541,4.453542,5.724616
2,0.789532,0.791874,0.875774,0.81906,7.299129,8.031186,5.428084,6.919466
3,0.694904,0.76197,0.840604,0.765826,8.671574,8.590015,6.14941,7.803666
4,0.599541,0.742917,0.815328,0.719262,9.836686,8.928354,6.619487,8.461509
5,0.509462,0.725572,0.798982,0.678006,10.780655,9.225573,6.906989,8.971072
6,0.42768,0.703718,0.788859,0.640085,11.532574,9.585781,7.080013,9.399456
7,0.354687,0.678857,0.783069,0.605538,12.216274,9.979096,7.178498,9.791289
8,0.289521,0.654197,0.777968,0.573895,12.833254,10.353984,7.264712,10.15065


csv file is saved to: ./result/40_10_1_50_1_50_datt_seq2seq_gru_exp_2_10.csv


10th iteration
att_type: exp
delta: 2.5
Epoch 1/10000
568/568 - 8s - loss: 0.4399 - val_loss: 0.2834 - 8s/epoch - 14ms/step
Epoch 2/10000
568/568 - 5s - loss: 0.2554 - val_loss: 0.2526 - 5s/epoch - 8ms/step
Epoch 3/10000
568/568 - 4s - loss: 0.2310 - val_loss: 0.2355 - 4s/epoch - 8ms/step
Epoch 4/10000
568/568 - 4s - loss: 0.2082 - val_loss: 0.2083 - 4s/epoch - 8ms/step
Epoch 5/10000
568/568 - 4s - loss: 0.1914 - val_loss: 0.1976 - 4s/epoch - 8ms/step
Epoch 6/10000
568/568 - 4s - loss: 0.1809 - val_loss: 0.1840 - 4s/epoch - 8ms/step
Epoch 7/10000
568/568 - 4s - loss: 0.1705 - val_loss: 0.1858 - 4s/epoch - 8ms/step
Epoch 8/10000
568/568 - 4s - loss: 0.1642 - val_loss: 0.1856 - 4s/epoch - 8ms/step
Epoch 9/10000
568/568 - 4s - loss: 0.1584 - val_loss: 0.1696 - 4s/epoch - 8ms/step
Epoch 10/10000
568/568 - 4s - loss: 0.1537 - val_loss: 0.1607 - 4s/epoch - 8ms/step
Epoch 11/10000
568/568 - 4s - loss: 0.1480 - val_

,TT-3061-3,TT-3061-5,LT-3061-2,mean,TT-3061-3,TT-3061-5,LT-3061-2,mean
index,R2,R2,R2,R2,nRMSE,nRMSE,nRMSE,nRMSE
0,0.928596,0.920068,0.958053,0.935572,4.336379,4.976154,3.153372,4.155302
1,0.864763,0.834093,0.917369,0.872075,5.931032,7.169861,4.426412,5.842435
2,0.767949,0.783412,0.859975,0.803779,7.664256,8.192812,5.762927,7.206665
3,0.663057,0.752681,0.821478,0.745738,9.112937,8.756023,6.507894,8.125618
4,0.562423,0.729234,0.803108,0.698255,10.282461,9.162878,6.83499,8.760109
5,0.472681,0.705969,0.79638,0.658343,11.177528,9.549403,6.951553,9.226161
6,0.394728,0.682234,0.794648,0.62387,11.859927,9.927239,6.982275,9.589814
7,0.329599,0.65703,0.794321,0.59365,12.451484,10.312647,6.989852,9.917994
8,0.279322,0.628281,0.790855,0.566153,12.925037,10.734956,7.050737,10.23691


csv file is saved to: ./result/40_10_1_50_1_50_datt_seq2seq_gru_exp_2.5_10.csv


end opitmization
